# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902037


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:28,  3.05s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:12,  2.60s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:50,  1.88s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:40,  1.58s/it]

...

Rendering models:  87%|████████▋ | 27/31 [00:23<00:02,  1.61it/s]

Rendering models:  90%|█████████ | 28/31 [00:23<00:01,  1.59it/s]

Rendering models:  94%|█████████▎| 29/31 [00:24<00:01,  1.98it/s]

Rendering models:  97%|█████████▋| 30/31 [00:24<00:00,  2.02it/s]

equidad1                              0.023437
not-logged-in-cae133427d99aac1e554    0.000423
kayleebug2017                         0.000710
not-logged-in-8c0c44faf754d344b18a    0.002515
tingard                               0.000181
Wain-PalmerM                          0.000697
Thedunkmasta                          0.002207
not-logged-in-ab1612bc18cd7aab85db    0.015373
not-logged-in-ea8ed61a7274cfb6a43b    0.013549
not-logged-in-ea8ed61a7274cfb6a43b    0.344343
not-logged-in-ea8ed61a7274cfb6a43b    0.002361
Vbgil13                               0.000971
not-logged-in-a0164d241f30411408f0    0.001081
not-logged-in-9cdd68bb509752635e0b    0.000855
jmartinez468                          0.138072
pangeli5                              0.000360
JulieHedglin                          0.000228
not-logged-in-59096a83d89ab5a9471b    0.001533
AlexYoung35                           0.000258
acapirala                             0.000733
not-logged-in-04ed9ae880f18762cd9e    0.022421
Lavadude     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())